In [6]:
import pandas as pd
import os

In [7]:
# 设置工作目录
data_dir = r'c:/Users/Administrator/Desktop/AI_code/Homework/HW3/data'
os.chdir(data_dir)

In [8]:
# 创建数据帧来存储合并后的销售和租赁数据
merged_sale = pd.DataFrame()
merged_rental = pd.DataFrame()

In [9]:
# 位置映射字典
block_mapping = {
    'centralpark': 'Central Park',
    'dazhulin': 'Dazhulin',
    'konggang': 'Konggang',
    'longtousi': 'Longtousi'
}

In [10]:
# 将数据转换为字符串的辅助函数
def convert_to_string(value):
    if pd.isna(value):
        return 'N/A'
    elif isinstance(value, (int, float)):
        return str(value)
    else:
        return str(value).strip()

In [11]:
# 遍历data文件夹中的所有parquet文件
for file in os.listdir('.'):
    if file.endswith('.parquet'):
        try:
            # 读取文件
            df = pd.read_parquet(file)
            print(f"处理文件: {file}")
            print(f"文件形状: {df.shape}")
            print(f"列名: {df.columns.tolist()}")
            
            # 确定文件类型（销售或租赁）
            file_lower = file.lower()
            is_sale = 'sale' in file_lower or 'price' in file_lower
            is_rental = 'rental' in file_lower or 'rent' in file_lower
            
            # 确定位置
            block = None
            for key, value in block_mapping.items():
                if key in file_lower:
                    block = value
                    break
            
            if block is None:
                print(f"警告: 无法从文件名 {file} 确定位置")
                continue
            
            # 添加block列
            df['Block'] = block
            
            # 提取所需的列
            if is_sale and is_rental:
                print(f"警告: 文件 {file} 同时包含销售和租赁关键词，跳过")
                continue
            elif is_sale:
                # 对于销售数据，提取面积和价格列（支持中英文列名）
                # 查找面积列
                area_col = None
                for col in df.columns:
                    if col.lower() in ['area', '面积'] or '㎡' in col:
                        area_col = col
                        break
                
                # 查找价格列
                price_col = None
                for col in df.columns:
                    col_lower = col.lower()
                    if any(keyword in col_lower for keyword in ['price', '价格', '总价']):
                        price_col = col
                        break
                
                if area_col and price_col:
                    # 确保Area和Price列是字符串类型，然后处理
                    sale_df = df[[area_col, price_col, 'Block']].copy()
                    sale_df.columns = ['Area', 'Price', 'Block']  # 标准化列名
                    
                    # 数据处理：根据位置进行特定处理
                    # 1. 价格单位转换：CentralPark和Longtousi的价格从万元转为元
                    # 2. Dazhulin的面积保留两位小数
                    
                    # 先尝试将Price列转换为数值类型进行计算
                    def convert_price(value, block):
                        try:
                            num_value = float(value)
                            if block in ['Central Park', 'Longtousi']:
                                # 万元转元
                                return str(int(num_value * 10000))
                            else:
                                # 其他位置保持原值
                                return str(int(num_value)) if '.' not in str(num_value) else str(num_value)
                        except (ValueError, TypeError):
                            return 'N/A'
                    
                    def format_area(value, location):
                        try:
                            num_value = float(value)
                            if location == 'Dazhulin':
                                # Dazhulin的面积保留两位小数
                                return f"{num_value:.2f}"
                            else:
                                # 其他位置保持原值
                                return str(num_value)
                        except (ValueError, TypeError):
                            return 'N/A'
                    
                    # 应用处理逻辑
                    sale_df['Price'] = sale_df.apply(lambda row: convert_price(row['Price'], row['Block']), axis=1)
                    sale_df['Area'] = sale_df.apply(lambda row: format_area(row['Area'], row['Block']), axis=1)
                    
                    # 确保所有列都是字符串类型
                    sale_df['Block'] = sale_df['Block'].apply(str)
                    
                    merged_sale = pd.concat([merged_sale, sale_df], ignore_index=True)
                    print(f"已添加到销售数据，当前合并后形状: {merged_sale.shape}")
                else:
                    print(f"警告: 销售文件 {file} 缺少面积或价格列")
            elif is_rental:
                # 对于租赁数据，提取面积和租金列（支持中英文列名）
                # 查找面积列
                area_col = None
                for col in df.columns:
                    if col.lower() in ['area', '面积'] or '㎡' in col:
                        area_col = col
                        break
                
                # 查找租金列
                rent_col = None
                for col in df.columns:
                    if col.lower() in ['rent', 'rent（元/月）', '月租金', '价格（元/月）']:
                        rent_col = col
                        break
                
                if area_col and rent_col:
                    # 确保Area和Rent列是字符串类型，然后处理
                    rental_df = df[[area_col, rent_col, 'Block']].copy()
                    rental_df.columns = ['Area', 'Rent', 'Block']  # 标准化列名
                    
                    # 转换所有数据为字符串
                    rental_df['Area'] = rental_df['Area'].apply(convert_to_string)
                    rental_df['Rent'] = rental_df['Rent'].apply(convert_to_string)
                    rental_df['Block'] = rental_df['Block'].apply(convert_to_string)
                    
                    merged_rental = pd.concat([merged_rental, rental_df], ignore_index=True)
                    print(f"已添加到租赁数据，当前合并后形状: {merged_rental.shape}")
                else:
                    print(f"警告: 租赁文件 {file} 缺少面积或租金列")
            else:
                print(f"警告: 无法确定文件 {file} 的类型（销售或租赁）")
        except Exception as e:
            print(f"处理文件 {file} 时出错: {e}")

# 保存合并后的数据
save_dir = r'c:/Users/Administrator/Desktop/AI_code/Homework/HW3'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

if not merged_sale.empty:
    sale_output_path = os.path.join(save_dir, 'merged_sale_data.parquet')
    merged_sale.to_parquet(sale_output_path, index=False)
    print(f"\n销售数据已合并并保存到: {sale_output_path}")
    print(f"合并后的销售数据形状: {merged_sale.shape}")
    print("销售数据前5行:")
    print(merged_sale.head())
else:
    print("\n没有找到有效的销售数据进行合并")

if not merged_rental.empty:
    rental_output_path = os.path.join(save_dir, 'merged_rental_data.parquet')
    merged_rental.to_parquet(rental_output_path, index=False)
    print(f"\n租赁数据已合并并保存到: {rental_output_path}")
    print(f"合并后的租赁数据形状: {merged_rental.shape}")
    print("租赁数据前5行:")
    print(merged_rental.head())
else:
    print("\n没有找到有效的租赁数据进行合并")

处理文件: CentralPark_rental_data_revised.parquet
文件形状: (1200, 6)
列名: ['Title', 'Area Info', 'Location', 'Subway Info', 'Rent', 'Area']
已添加到租赁数据，当前合并后形状: (1200, 3)
处理文件: CentralPark_sale_data_revised.parquet
文件形状: (1200, 6)
列名: ['Title', 'Area Info', 'Location', 'Subway Info', 'Price', 'Area']
已添加到销售数据，当前合并后形状: (1200, 3)
处理文件: dazhulin_rent.parquet
文件形状: (415, 5)
列名: ['标题', '面积', '月租金', '每平米月租金', '房屋信息']
已添加到租赁数据，当前合并后形状: (1615, 3)
处理文件: dazhulin_sale.parquet
文件形状: (1200, 7)
列名: ['标题', '总价', '单价', '面积', '房屋信息', '地址', '区域']
已添加到销售数据，当前合并后形状: (2400, 3)
处理文件: konggang_housing_price.parquet
文件形状: (1200, 10)
列名: ['标题', '户型', '面积（㎡）', '楼层', '朝向', '年份', '户主', '位置', '价格（元）', 'block']
已添加到销售数据，当前合并后形状: (3600, 3)
处理文件: konggang_housing_rent.parquet
文件形状: (499, 7)
列名: ['标题', '户型', '面积（㎡）', '朝向', '位置', '价格（元/月）', 'block']
已添加到租赁数据，当前合并后形状: (2114, 3)
处理文件: Longtousi_rental_data_20pages.parquet
文件形状: (1200, 6)
列名: ['Title', 'Area Info', 'Location', 'Subway Info', 'Rent', 'Area']
已添加到租赁数据，当前合并后形状: (3314,